In [1]:
import pickle
import numpy
from PIL import Image
import IPython.display
import os

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention
import tracking.GaussianGenerator

/home/fmpaezri/.local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [3]:
with open('/home/fmpaezri/repos/localization-agent/model.pkl0', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 11.3 s, sys: 364 ms, total: 11.7 s
Wall time: 11.9 s


In [4]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [5]:
tester = tracking.Tester.Tester(tracker)

In [7]:
samplesDir = '/tmp/sampleSequences'

gg = tracking.GaussianGenerator.GaussianGenerator(
    '/home/fmpaezri/datasets/mscoco/',
    '/home/fmpaezri/repos/localization-agent/summaries/cocoTrainSummaryCategAndSideGt100SmplsAllCorrected.pkl',
    ['random', 'sine', 'stretched'], ['stretched'], None, grayscale=False, 
    objectPathTemplate='./train2014', scenePathTemplate='./train2014'
        )

%time gg.saveBatch(4, 0, 60, samplesDir)

Loading summary from file /home/fmpaezri/repos/localization-agent/summaries/cocoTrainSummaryCategAndSideGt100SmplsAllCorrected.pkl
CPU times: user 456 ms, sys: 204 ms, total: 660 ms
Wall time: 1.63 s


In [10]:
#sequencePath = '/home/fmpaezri/datasets/vot-challenge/sequences2014/basketball/'
sequencePath = os.path.join(samplesDir, numpy.random.choice(
        [sequence for sequence in os.listdir(samplesDir) if os.path.isdir(os.path.join(samplesDir, sequence))]
    ))

In [11]:
%time data, labels = tester.loadImageSequence(sequencePath, newShape=(rnn.imgSize,rnn.imgSize))

CPU times: user 32 ms, sys: 4 ms, total: 36 ms
Wall time: 37.3 ms


In [ ]:
#ONLY FOR VOT BOXES
#Correcting box ccords as expected by IoU
#labelsCopy = labels.copy()
#labelsCopy[:,0] = labels[:,0::2].min(axis=1) 
#labelsCopy[:,1] = labels[:,1::2].min(axis=1)
#labelsCopy[:,2] = labels[:,0::2].max(axis=1) 
#labelsCopy[:,3] = labels[:,1::2].max(axis=1)
#labels = labelsCopy

In [12]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [13]:
labels = labels[numpy.newaxis,...]

In [14]:
print data.shape, labels.shape

(1, 60, 100, 100, 3) (1, 60, 4)


In [15]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, labels.shape[1], 4, '/tmp')

CPU times: user 3.26 s, sys: 152 ms, total: 3.42 s
Wall time: 1.83 s


In [16]:
iou.mean()

0.16722241064334778

In [17]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [18]:
seq.addBoxes(labels[0], 'green')
seq.addBoxes((predLabels[0]), 'red')

In [19]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [20]:
IPython.display.HTML(vs.displayHTML(outputName))